# AI Atlas Explorer: Generic Graph Navigation

This notebook demonstrates the **AI Atlas Explorer** - a generic graph navigation system for the AI Atlas Nexus knowledge graph.

**What you'll learn:**
- How to create and use AtlasExplorer for graph navigation
- Different ways to query the knowledge graph (basic, pattern-based, custom)
- How to navigate multi-hop relationships
- Understanding navigation statistics and results

**Key Features:**
- ✅ Generic BFS (Breadth-First Search) traversal
- ✅ Named navigation patterns for common queries
- ✅ Custom configuration for complex traversals
- ✅ Full navigation statistics and transparency

## 1. Setup and Initialization

In [ ]:
from ai_atlas_nexus import AIAtlasNexus
from ai_atlas_nexus.blocks.ai_atlas_explorer import (
    AtlasExplorer,
    GraphEntityType,
    GraphRelationType,
    NavigationConfig
)

## 2. Initialize AI Atlas Nexus and Create Explorer

In [ ]:
# Initialize AI Atlas Nexus
ran = AIAtlasNexus()

# Create AtlasExplorer instance (no base_dir needed - mappings are in ontology)
explorer = AtlasExplorer(ran._ontology)

print("✓ AI Atlas Nexus initialized")
print(f"  Risks: {len(ran._ontology.risks or [])}")
print(f"  Capabilities: {len(ran._ontology.capabilities or [])}")
print(f"  Tasks: {len(ran._ontology.aitasks or [])}")
print(f"  Intrinsics: {len(ran._ontology.llmintrinsics or [])}")
print(f"  Adapters: {len(ran._ontology.adapters or [])}")
print("\n✓ AtlasExplorer ready for navigation")

## 3. Basic Entity Retrieval

The simplest operations: getting all entities of a type or finding a specific entity.

In [ ]:
# Get all capabilities
capabilities = explorer.get_all_capabilities()

print(f"Total Capabilities: {len(capabilities)}\n")
for cap in capabilities[:5]:  # Show first 5
    print(f"• {cap.name}")
    print(f"  ID: {cap.id}")
    print(f"  Group: {cap.isPartOf}")
    print()

In [ ]:
# Get a specific capability by ID
cap = explorer.get_capability(id="ibm-cap-reading-comprehension")

if cap:
    print(f"Found Capability: {cap.name}")
    print(f"  ID: {cap.id}")
    if cap.description:
        print(f"  Description: {cap.description[:150]}...")
else:
    print("Capability not found")

In [ ]:
# Get all tasks
tasks = explorer.get_all_tasks()

print(f"\nTotal AI Tasks: {len(tasks)}\n")
for task in tasks:
    print(f"• {task.name} ({task.id})")

## 4. Capability-Task Navigation

Navigate between capabilities and tasks using the lifted TSV mappings.

In [ ]:
# Find capabilities required for a specific task
task_capabilities = explorer.get_capabilities_for_task(task_id="question-answering")

print("Question Answering task requires these capabilities:\n")
for cap in task_capabilities:
    print(f"  • {cap.name}")
    print(f"    ID: {cap.id}")

print(f"\n✓ Found {len(task_capabilities)} required capabilities")

In [ ]:
# Find tasks that require a specific capability
capability_tasks = explorer.get_tasks_for_capability(
    capability_id="ibm-cap-reading-comprehension"
)

print("Reading Comprehension capability is required by:\n")
for task in capability_tasks:
    print(f"  • {task.name} ({task.id})")

print(f"\n✓ Found {len(capability_tasks)} tasks")

## 5. Capability-Intrinsic Navigation

Find which intrinsics and adapters implement specific capabilities.

In [ ]:
# Find intrinsics that implement a capability
intrinsics = explorer.get_intrinsics_for_capability(
    capability_id="ibm-cap-reading-comprehension"
)

print("Reading Comprehension is implemented by:\n")
for intr in intrinsics:
    print(f"  • {intr.name}")
    print(f"    ID: {intr.id}")

print(f"\n✓ Found {len(intrinsics)} implementations")

## 6. End-to-End Tracing

Trace from task → capabilities → intrinsics in a single call.

In [ ]:
# Complete trace from task to implementations
trace_result = explorer.trace_task_to_intrinsics(task_id="question-answering")

print("Complete Trace: Question Answering → Capabilities → Intrinsics\n")
print(f"Task: {trace_result['task'].name}")
print(f"\nRequired Capabilities: {len(trace_result['capabilities'])}")

for cap in trace_result['capabilities']:
    print(f"\n  • {cap.name}")
    
    cap_intrinsics = trace_result['intrinsics_by_capability'].get(cap.id, [])
    if cap_intrinsics:
        print(f"    Implementations: {len(cap_intrinsics)}")
        for intr in cap_intrinsics[:2]:  # Show first 2
            print(f"      - {intr.name}")
        if len(cap_intrinsics) > 2:
            print(f"      ... and {len(cap_intrinsics) - 2} more")

print(f"\n✓ Total intrinsics across all capabilities: {len(trace_result['all_intrinsics'])}")

## 7. Risk Navigation Examples

Working with the risk portion of the knowledge graph.

In [ ]:
# Get all risks
risks = explorer.get_all_risks()

print(f"Total Risks in Knowledge Graph: {len(risks)}")
print("\nRisks by Taxonomy:")

# Count by taxonomy
from collections import Counter
taxonomy_counts = Counter([r.isDefinedByTaxonomy for r in risks if hasattr(r, 'isDefinedByTaxonomy')])

for taxonomy, count in taxonomy_counts.most_common(5):
    print(f"  {taxonomy}: {count} risks")

In [ ]:
# Get a specific risk
risk = explorer.get_risk(id="atlas-harmful-output")

if risk:
    print(f"Risk: {risk.name}")
    print(f"  Taxonomy: {risk.isDefinedByTaxonomy}")
    if risk.description:
        print(f"  Description: {risk.description[:200]}...")
else:
    print("Risk not found")

## 8. Navigation with Named Patterns

AtlasExplorer provides pre-configured patterns for common navigation tasks.

In [ ]:
# Navigate using the 'related_risks' pattern
if risk:
    result = explorer.navigate(
        risk.id,
        GraphEntityType.RISK,
        pattern="related_risks"
    )
    
    related_risks = result.get_nodes_by_type(GraphEntityType.RISK)
    
    print(f"Starting from: {risk.name}")
    print(f"\nNavigation Statistics:")
    print(f"  Total nodes visited: {result.statistics['nodes_visited']}")
    print(f"  Nodes returned: {result.statistics['nodes_returned']}")
    print(f"  Max depth reached: {result.statistics['max_depth_reached']}")
    print(f"  Relationships traversed: {result.statistics['relationships_traversed']}")
    
    print(f"\nRelated Risks Found: {len(related_risks)}")
    for node in related_risks[:3]:  # Show first 3
        r = next((r for r in risks if r.id == node.entity_id), None)
        if r:
            print(f"  • {r.name} (depth={node.depth})")

## 9. Custom Navigation Configuration

For complete control, create a custom NavigationConfig.

In [ ]:
# Create custom navigation config
config = NavigationConfig(
    max_depth=2,  # Two-hop traversal
    included_relationships=[
        GraphRelationType.EXACT_MATCH,
        GraphRelationType.CLOSE_MATCH,
        GraphRelationType.BROAD_MATCH,
        GraphRelationType.NARROW_MATCH,
    ],
    included_entity_types=[
        GraphEntityType.RISK,
    ],
    deduplicate_results=True,
    cache_enabled=True
)

# Navigate with custom config
if risk:
    result = explorer.navigate(
        risk.id,
        GraphEntityType.RISK,
        config=config
    )
    
    print("Custom Navigation with 2-hop SKOS relationships:")
    print(f"  Nodes visited: {result.statistics['nodes_visited']}")
    print(f"  Depth reached: {result.statistics['max_depth_reached']}")
    
    # Analyze by depth
    print("\nNodes by Depth:")
    for depth in range(3):
        nodes_at_depth = result.get_nodes_at_depth(depth)
        if nodes_at_depth:
            print(f"  Depth {depth}: {len(nodes_at_depth)} nodes")

## 10. Multi-Hop Navigation Example

Navigate through multiple relationship types to find indirect connections.

In [ ]:
# Example: Risk → Related Risks → Risk Controls (2-hop)
config = NavigationConfig(
    max_depth=2,
    included_relationships=[
        # SKOS relationships
        GraphRelationType.EXACT_MATCH,
        GraphRelationType.CLOSE_MATCH,
        GraphRelationType.BROAD_MATCH,
        GraphRelationType.NARROW_MATCH,
        # Control relationship
        GraphRelationType.IS_DETECTED_BY,
    ],
    included_entity_types=[
        GraphEntityType.RISK,
        GraphEntityType.RISK_CONTROL,
    ],
)

if risk:
    result = explorer.navigate(
        risk.id,
        GraphEntityType.RISK,
        config=config
    )
    
    controls = result.get_nodes_by_type(GraphEntityType.RISK_CONTROL)
    
    print("Multi-Hop Navigation: Risk → Related Risks → Controls")
    print(f"\nStarting risk: {risk.name}")
    print(f"\nControls found (direct + via related risks): {len(controls)}")
    
    print("\nNavigation Path Analysis:")
    for depth in range(3):
        nodes = result.get_nodes_at_depth(depth)
        if nodes:
            risks_at_depth = [n for n in nodes if n.entity_type == GraphEntityType.RISK]
            controls_at_depth = [n for n in nodes if n.entity_type == GraphEntityType.RISK_CONTROL]
            print(f"  Depth {depth}:")
            if risks_at_depth:
                print(f"    Risks: {len(risks_at_depth)}")
            if controls_at_depth:
                print(f"    Controls: {len(controls_at_depth)}")

## 11. Convenience Methods vs Generic Navigation

AtlasExplorer provides both convenience methods and generic navigation.

In [ ]:
# Approach 1: Convenience method (simple)
related_risks_convenience = explorer.get_related_risks(risk=risk) if risk else []

print("Approach 1 - Convenience Method:")
print(f"  Related risks: {len(related_risks_convenience)}")
print("  Pros: Simple, one line of code")
print("  Cons: No visibility into navigation process\n")

# Approach 2: Generic navigation with named pattern (visibility)
if risk:
    result = explorer.navigate(
        risk.id,
        GraphEntityType.RISK,
        pattern="related_risks"
    )
    related_risks_generic = result.get_nodes_by_type(GraphEntityType.RISK)
    
    print("Approach 2 - Generic Navigation:")
    print(f"  Related risks: {len(related_risks_generic)}")
    print(f"  Navigation statistics: {result.statistics}")
    print("  Pros: Full transparency, statistics, customizable")
    print("  Cons: Slightly more verbose")

## 12. Summary

### What We Learned:

✅ **Basic Retrieval**: `get_all_*()` and `get_*()` methods for simple queries

✅ **Capability-Task Navigation**: Query which capabilities tasks require

✅ **Capability-Intrinsic Navigation**: Find implementations for capabilities

✅ **End-to-End Tracing**: Complete task → capability → intrinsic paths

✅ **Named Patterns**: Pre-configured navigation patterns like `"related_risks"`

✅ **Custom Configuration**: Full control with `NavigationConfig`

✅ **Multi-Hop Navigation**: Complex traversals through multiple relationship types

✅ **Statistics & Transparency**: Understanding what the navigator is doing

### When to Use Each Approach:

| Need | Approach | Example |
|------|----------|----------|
| Simple query | Convenience method | `explorer.get_all_capabilities()` |
| Task → Capabilities | Convenience method | `explorer.get_capabilities_for_task(task_id)` |
| Common pattern + stats | Named pattern | `explorer.navigate(..., pattern="related_risks")` |
| Complex/custom query | Custom config | `explorer.navigate(..., config=config)` |
| Multi-hop traversal | Custom config | Set `max_depth=2+` and specify relationships |

### Key Benefits:

- **Generic**: Works with any entity type and relationship
- **Transparent**: Full visibility into navigation process
- **Flexible**: From simple convenience methods to complex custom queries
- **Performant**: BFS with caching and deduplication
- **Complete**: All mappings (task↔capability, capability↔intrinsic) integrated
- **Extensible**: Easy to add new entity types and patterns

---

**For more information:**
- Capabilities notebook: `ai_capabilities_identification.ipynb`
- AI Atlas Nexus: https://github.com/IBM/ai-atlas-nexus